In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog, simpledialog
from datetime import datetime
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook
from openpyxl.styles import Font
from openpyxl.formatting.rule import CellIsRule


# === Step 1: Select file ===
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename(title="Select ADWP Data File", filetypes=[("Excel files", "*.xlsx")])
if not file_path:
    raise Exception("❌ No file selected!")

# === Step 2: Year & Month selection ===
year = simpledialog.askinteger("Select Year", "Enter base year (e.g., 2025):")
month = simpledialog.askinteger("Select Month", "Enter base month number (1-12):")
if not year or not month:
    raise Exception("❌ Year/Month not selected!")

selected_period = datetime(year, month, 1)

# === Step 3: Load Data ===
df = pd.read_excel(file_path, sheet_name="Data")

# Ensure Period Month is datetime
df["Period Month"] = pd.to_datetime(df["Period Month"])

# Horizon: 24 months
horizon = [selected_period + relativedelta(months=i) for i in range(24)]
horizon_labels = [d.strftime("%b-%y") for d in horizon]


# === Step 4: Helper to snapshot FTE ===
def get_snapshot_fte(series, sel_period):
    """Return the FTE value as of selected period, even if it starts later."""
    if sel_period in series.index:
        return series.loc[sel_period]
    future_periods = [p for p in series.index if p > sel_period]
    if future_periods:
        return series.loc[future_periods[0]]
    past_periods = [p for p in series.index if p < sel_period]
    if past_periods:
        return series.loc[past_periods[-1]]
    return None


# === Step 5: Process Data ===
records = []
grouped = df.groupby(
    df.apply(lambda r: (r["PID"], r["GCB"], r["Business Framework"]) 
             if pd.notna(r["PID"]) 
             else (r["MPP ID"], r["GCB"], r["Business Framework"]), axis=1)
)

for _, group in grouped:
    # Split rows if BF changes mid-way
    bf_groups = group.groupby("Business Framework")

    for bf, bf_group in bf_groups:
        s = bf_group.set_index("Period Month")["FTE"].sort_index()

        record = {
            "Business Service L2": bf_group["Business Service L2"].iloc[0],
            "Business Framework Group": bf_group["Business Framework Group"].iloc[0],
            "Business Framework": bf,
            "Country R1": bf_group["Country R1"].iloc[0],
            "Country R2": bf_group["Country R2"].iloc[0],
            "Country R3": bf_group["Country R3"].iloc[0],
            "PID": bf_group["PID"].iloc[0] if pd.notna(bf_group["PID"].iloc[0]) else None,
            "GCB": bf_group["GCB"].iloc[0],
            "MPP ID": bf_group["MPP ID"].iloc[0] if pd.isna(bf_group["PID"].iloc[0]) else None,
            "Stack": bf_group["Stack"].iloc[0] if pd.isna(bf_group["PID"].iloc[0]) else None,
        }

        last_fte = None
        for d, label in zip(horizon, horizon_labels):
            if d in s.index:
                last_fte = s.loc[d]
                record[label] = last_fte
            else:
                record[label] = last_fte

        # Snapshot FTE
        record["FTE"] = get_snapshot_fte(s, selected_period)

        records.append(record)


# === Step 6: Build Output ===
out_df = pd.DataFrame(records)

# Reorder columns
base_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "GCB",
    "FTE", "MPP ID", "Stack"
]
final_cols = base_cols + horizon_labels
out_df = out_df[final_cols]


# === Step 7: Save Output ===
out_file = file_path.replace(".xlsx", "_ADWP_Output.xlsx")
out_df.to_excel(out_file, sheet_name="Output", index=False)

# === Step 8: Highlight negative numbers in red font ===
wb = load_workbook(out_file)
ws = wb["Output"]

max_row = ws.max_row
max_col = ws.max_column
col_letters = [ws.cell(row=1, column=i).column_letter for i in range(1, max_col + 1)]

red_font = Font(color="FF0000")

for col_letter in col_letters:
    ws.conditional_formatting.add(
        f"{col_letter}2:{col_letter}{max_row}",
        CellIsRule(operator="lessThan", formula=["0"], font=red_font)
    )

wb.save(out_file)
print(f"✅ ADWP processing complete. File saved: {out_file}")


In [ ]:
import pandas as pd
import calendar
from datetime import datetime
from tkinter import Tk, filedialog, StringVar, OptionMenu, Button, Label, Toplevel
import os

# -------------------------
# Step 1. File selection
# -------------------------
root = Tk()
root.withdraw()  # hide root window

file_path = filedialog.askopenfilename(
    title="Select ADWP Data File",
    filetypes=[("Excel files", "*.xlsx")]
)
if not file_path:
    raise SystemExit("❌ No file selected.")
print(f"Selected file: {file_path}")


# -------------------------
# Step 2. Custom dialog for Year & Month
# -------------------------
def ask_year_month():
    result = {"year": None, "month": None}

    def on_ok():
        result["year"] = year_var.get()
        result["month"] = month_var.get()
        win.destroy()

    win = Toplevel(root)
    win.title("Select Year and Month")

    Label(win, text="Select Year:").grid(row=0, column=0, padx=10, pady=5)
    Label(win, text="Select Month:").grid(row=1, column=0, padx=10, pady=5)

    # Year dropdown
    current_year = datetime.today().year
    years = [str(current_year - 1), str(current_year), str(current_year + 1)]
    year_var = StringVar(win)
    year_var.set(str(current_year))  # default
    year_menu = OptionMenu(win, year_var, *years)
    year_menu.grid(row=0, column=1, padx=10, pady=5)

    # Month dropdown
    months = list(calendar.month_name)[1:]
    month_var = StringVar(win)
    month_var.set(calendar.month_name[datetime.today().month])  # default
    month_menu = OptionMenu(win, month_var, *months)
    month_menu.grid(row=1, column=1, padx=10, pady=5)

    Button(win, text="OK", command=on_ok).grid(row=2, column=0, columnspan=2, pady=10)

    win.wait_window()  # wait for user to close

    return int(result["year"]), result["month"]


selected_year, selected_month = ask_year_month()
print(f"Selected: {selected_month} {selected_year}")


# -------------------------
# Step 3. Processing
# -------------------------
print("⏳ Reading input file...")
df = pd.read_excel(file_path, sheet_name="Data", engine="openpyxl")
df["Period Month"] = pd.to_datetime(df["Period Month"]).dt.to_period("M").dt.to_timestamp()

# Current selection
sel_year = selected_year
sel_month = list(calendar.month_name).index(selected_month)
curr_date = datetime(sel_year, sel_month, 1)

# Horizon (24 months from Jan of selected year)
start_date = datetime(sel_year, 1, 1)
horizon = pd.date_range(start=start_date, periods=24, freq="MS")
month_labels = [d.strftime("%b-%y") for d in horizon]

output_records = []

def get_key(row):
    if pd.notna(row["PID"]):
        return ("PID", row["PID"], row["GCB"], row["Business Framework"])
    else:
        return ("MPP", row["MPP ID"], row["GCB"], row["Business Framework"])

print("⏳ Processing records...")
grouped = df.groupby(df.apply(get_key, axis=1))

for key, group in grouped:
    kind = key[0]
    group = group.sort_values("Period Month")

    latest = group.iloc[-1]
    record = {
        "Business Service L2": latest.get("Business Service L2"),
        "Business Framework Group": latest.get("Business Framework Group"),
        "Business Framework": latest.get("Business Framework"),
        "Country R1": latest.get("Country R1"),
        "Country R2": latest.get("Country R2"),
        "Country R3": latest.get("Country R3"),
        "PID": latest.get("PID") if kind == "PID" else None,
        "GCB": latest.get("GCB"),
        "MPP ID": latest.get("MPP ID") if kind == "MPP" else None,
        "Stack": latest.get("Stack") if kind == "MPP" else None
    }

    # initialize horizon columns
    for label in month_labels:
        record[label] = None

    last_fte = None
    for month in month_labels:
        month_dt = datetime.strptime(month, "%b-%y")
        row = group[group["Period Month"] == month_dt]
        if not row.empty:
            last_fte = row["FTE"].iloc[0]
        if last_fte is not None:
            record[month] = last_fte
        if month_dt <= curr_date and record[month] is not None:
            record["FTE"] = record[month]

    output_records.append(record)

# Build DataFrame
out_df = pd.DataFrame(output_records)

# Sorting
sort_cols = [
    "Business Service L2", "Business Framework Group", "Business Framework",
    "Country R1", "Country R2", "Country R3", "PID", "MPP ID", "GCB"
]
out_df = out_df.sort_values(sort_cols, na_position="last").reset_index(drop=True)

# Save output
out_file = os.path.join(
    os.path.dirname(file_path),
    f"ADWP_Output_{sel_year}_{sel_month:02}.xlsx"
)
with pd.ExcelWriter(out_file, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Output", index=False)

print(f"✅ ADWP Output written to: {out_file}")
print(f"Rows: {len(out_df):,}, Columns: {len(out_df.columns)}")
